In [2]:
import cv2
import mediapipe as mp
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from collections import deque

# ============================================================
# VIDEO UPLOAD + processed video
# ============================================================


In [7]:
# =========================================================
# 1. INPUT VIDEO
# =========================================================
video_path = "/Users/louiscoussement/code/VERA/data/raw/myvideo.mp4"
output_path = "/Users/louiscoussement/code/VERA/data/processed/debug_pose_minimal.mp4"

cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    raise ValueError("❌ Error loading video")
print("✅ Video loaded")

fps = cap.get(cv2.CAP_PROP_FPS)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

out = cv2.VideoWriter(
    output_path,
    cv2.VideoWriter_fourcc(*'mp4v'),
    fps,
    (width, height)
)

# =========================================================
# 2. INITIALIZE MEDIAPIPE (Holistic = Pose + Hands)
# =========================================================
mp_holistic = mp.solutions.holistic
mp_drawing  = mp.solutions.drawing_utils

holistic = mp_holistic.Holistic(
    static_image_mode=False,
    model_complexity=1,
    enable_segmentation=False,
    refine_face_landmarks=False      # ❗️DISABLE FACE LANDMARKS
)

# =========================================================
# 3. LANDMARK GROUPS (COLOR-CODED)
# =========================================================
POSE_POINTS = {
    "shoulders": [11, 12],
    "hips":      [23, 24],
    "wrists":    [15, 16],
}

COLOR_SHOULDERS = (255, 0,   0)
COLOR_HIPS      = (0,   255, 255)
COLOR_WRISTS    = (0,   0, 255)

# =========================================================
# 4. PROCESS VIDEO FRAME BY FRAME + DRAW
# =========================================================
while True:
    ret, frame = cap.read()
    if not ret:
        break

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = holistic.process(rgb)

    annotated = frame.copy()
    h, w, _ = frame.shape

    if results.pose_landmarks:
        lm = results.pose_landmarks.landmark

        # ---- Custom colored keypoints ----
        for i in POSE_POINTS["shoulders"]:
            cv2.circle(annotated, (int(lm[i].x*w), int(lm[i].y*h)), 5, COLOR_SHOULDERS, -1)
        for i in POSE_POINTS["hips"]:
            cv2.circle(annotated, (int(lm[i].x*w), int(lm[i].y*h)), 5, COLOR_HIPS, -1)
        for i in POSE_POINTS["wrists"]:
            cv2.circle(annotated, (int(lm[i].x*w), int(lm[i].y*h)), 6, COLOR_WRISTS, -1)

        # ---- Full body skeleton (NO face) ----
        mp_drawing.draw_landmarks(
            annotated,
            results.pose_landmarks,
            mp_holistic.POSE_CONNECTIONS
        )

    out.write(annotated)

cap.release()
out.release()

print("🎥 Debug Pose video saved to:", output_path)


✅ Video loaded


I0000 00:00:1764773189.374402       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


🎥 Debug Pose video saved to: /Users/louiscoussement/code/VERA/data/processed/debug_pose_minimal.mp4


# ============================================================
# Initialize MediaPipe Holistic MODEL
# ============================================================

In [8]:
#mp_holistic = mp.solutions.holistic
#mp_drawing = mp.solutions.drawing_utils

# Arrays to store landmarks for later analysis
#pose_landmarks_all = []
#left_hand_landmarks_all = []
#right_hand_landmarks_all = []


====================================================
# 1 - FUNCTIONS
====================================================

### Define Torso center + body sway (torso speed)

In [9]:
def compute_torso_center(lm):
    L_sh = np.array([lm[11].x, lm[11].y, lm[11].z])
    R_sh = np.array([lm[12].x, lm[12].y, lm[12].z])
    L_hp = np.array([lm[23].x, lm[23].y, lm[23].z])
    R_hp = np.array([lm[24].x, lm[24].y, lm[24].z])
    return (L_sh + R_sh + L_hp + R_hp) / 4

def compute_body_sway(lm, prev_torso):
    torso = compute_torso_center(lm)

    if prev_torso is None:
        return np.nan, torso

    sway = np.linalg.norm(torso - prev_torso)
    return sway, torso

### Gesture magnitude

In [10]:
def compute_gesture_magnitude(lm):
    torso = compute_torso_center(lm)
    L_wr = np.array([lm[15].x, lm[15].y, lm[15].z])
    R_wr = np.array([lm[16].x, lm[16].y, lm[16].z])

    mag_L = np.linalg.norm(L_wr - torso)
    mag_R = np.linalg.norm(R_wr - torso)

    return np.nanmean([mag_L, mag_R])

### gesture activity rate (speed)

In [11]:
def compute_gesture_activity(lm, prev_L_wr, prev_R_wr):
    L_wr = np.array([lm[15].x, lm[15].y, lm[15].z])
    R_wr = np.array([lm[16].x, lm[16].y, lm[16].z])

    if prev_L_wr is None:
        return np.nan, L_wr, R_wr

    speed_L = np.linalg.norm(L_wr - prev_L_wr)
    speed_R = np.linalg.norm(R_wr - prev_R_wr)

    activity = np.nanmean([speed_L, speed_R])

    return activity, L_wr, R_wr

### Compute posture openness (shoulder angle)

In [12]:
def compute_posture_openness(lm):
    L_sh = np.array([lm[11].x, lm[11].y, lm[11].z])
    R_sh = np.array([lm[12].x, lm[12].y, lm[12].z])
    neck = (L_sh + R_sh) / 2

    v1 = L_sh - neck
    v2 = R_sh - neck

    dot = np.dot(v1, v2)
    norm = np.linalg.norm(v1) * np.linalg.norm(v2)
    if norm == 0:
        return np.nan

    angle = np.arccos(np.clip(dot / norm, -1, 1))
    return np.degrees(angle)

## Video metrics extraction

In [13]:
cap.release()
cap = cv2.VideoCapture(video_path)

features = []

prev_L_wr = None
prev_R_wr = None
prev_torso = None

fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

with mp_holistic.Holistic(model_complexity=1) as holistic:
    for idx in tqdm(range(frame_count)):
        ret, frame = cap.read()
        if not ret:
            break

        timestamp = idx / fps
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(rgb)

        annotated = frame.copy()
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                annotated, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS
            )
        out.write(annotated)

        if not results.pose_landmarks:
            features.append({
                "timestamp": timestamp,
                "gesture_magnitude": np.nan,
                "gesture_activity": np.nan,
                "body_sway": np.nan,
                "posture_openness": np.nan
            })
            continue

        lm = results.pose_landmarks.landmark

        # ----- METRIC 1 : Gesture Magnitude -----
        gesture_magnitude = compute_gesture_magnitude(lm)

        # ----- METRIC 2 : Gesture Activity -----
        gesture_activity, prev_L_wr, prev_R_wr = compute_gesture_activity(lm, prev_L_wr, prev_R_wr)

        # ----- METRIC 3 : Body Sway -----
        body_sway, prev_torso = compute_body_sway(lm, prev_torso)

        # ----- METRIC 4 : Posture Openness -----
        posture_openness = compute_posture_openness(lm)

        features.append({
            "timestamp": timestamp,
            "gesture_magnitude": gesture_magnitude,
            "gesture_activity": gesture_activity,
            "body_sway": body_sway,
            "posture_openness": posture_openness
        })


I0000 00:00:1764773298.339671       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M2
 98%|█████████▊| 1741/1774 [01:01<00:01, 28.36it/s]


====================================================
# 2 - GROUP TIMESTAMPS OF THE VIDEO PER SEC
====================================================

In [14]:
df = pd.DataFrame(features).set_index("timestamp")
df["second"] = df.index.astype(int)
df

,gesture_magnitude,gesture_activity,body_sway,posture_openness,second
timestamp,,,,,
0.000000,0.284515,NaN,NaN,180.000000,0
0.033378,0.410110,0.127347,0.014879,180.000000,0
0.066755,0.397581,0.016972,0.005097,179.999999,0
0.100133,0.430820,0.040172,0.006270,180.000000,0
0.133510,0.430281,0.026564,0.004433,179.999999,0
...,...,...,...,...,...
57.943420,0.562022,0.006043,0.001120,180.000000,57
57.976797,0.565191,0.003473,0.000197,180.000000,57
58.010175,0.560470,0.007328,0.000370,179.999999,58


In [15]:
gesture_mag_1s = df.groupby("second")["gesture_magnitude"].mean().fillna(0)
gesture_act_1s = df.groupby("second")["gesture_activity"].mean().fillna(0)
gesture_jitter_1s = df.groupby("second")["gesture_activity"].var().fillna(0)
body_sway_1s   = df.groupby("second")["body_sway"].var().fillna(0)
posture_open_1s = df.groupby("second")["posture_openness"].mean().fillna(0)

====================================================
# 4 - DISPLAY OBSERVATIONS ON A 5 SEC WINDOWS/1SEC SLICING
====================================================

In [16]:
def sliding_windows(series, window=5):
    rows = []
    seconds = series.index.values

    for start in seconds:
        end = start + window
        win = series.loc[start:end]

        if len(win) == window + 1:  # require full window
            rows.append({
                "start_sec": start,
                "end_sec": end,
                "value": win.mean()
            })

    return pd.DataFrame(rows)


In [17]:
df_mag_5s     = sliding_windows(gesture_mag_1s)
df_act_5s     = sliding_windows(gesture_act_1s)
df_jitter_5s  = sliding_windows(gesture_jitter_1s)
df_sway_5s    = sliding_windows(body_sway_1s)
df_open_5s    = sliding_windows(posture_open_1s)

====================================================
# 5 - SCORING
====================================================


## 5.1 - GESTURE MAGNITUDE (more = expressive = good)

In [18]:
# --- GESTURE MAGNITUDE ---
BASELINE_GESTURE_MAG_MEAN = 0.20
# 0.20 normalized distance = moderate expressiveness.

BASELINE_GESTURE_MAG_RANGE = 0.10
# Larger = more tolerance for variance.

In [19]:
# ========== GESTURE MAGNITUDE ==========
# RELATIVE score
z_mag = (df_mag_5s["value"] - df_mag_5s["value"].mean()) / df_mag_5s["value"].std()
df_mag_5s["rel_score"] = 1 / (1 + np.exp(-z_mag))

# ABSOLUTE score
mag_abs = df_mag_5s["value"].mean()
abs_mag_score = 1 / (1 + np.exp(-(mag_abs - BASELINE_GESTURE_MAG_MEAN) / BASELINE_GESTURE_MAG_RANGE))

# FINAL MAGNITUDE SCORE
global_gesture_magnitude = 0.5 * abs_mag_score + 0.5 * df_mag_5s["rel_score"].mean()
print(global_gesture_magnitude)

0.7152394905341581


## Gesture Magnitude:
The $\mathbf{global\_gesture\_magnitude}$ score emphasizes the overall size and consistency of the speaker's arm movements throughout the video, providing a single metric that represents their combined absolute expressiveness (how far gestures are from the body compared to a standard) and the temporal consistency of that size (how much the gesture size varies over time).



(How wide / open the hand gestures are)
Score = sigmoid(-z) → higher = better expressiveness


1. ≥ 0.55 — Strong expressive range (Excellent)

“Open, dynamic gestures that project clarity and energy.”
Wide, confident arm movements
Gestures match the speech rhythm
High perceived engagement and charisma
Often seen in TED speakers & sales presenters

2. 0.45 – 0.55 — Healthy expressive amplitude (Good)

“Balanced gestures — visible but not distracting.”
Natural arm extension
Helps structure the message
Neither too small nor overwhelming
Ideal for pitches, meetings, interviews

3. 0.30 – 0.45 — Under-expressive or tight range (Weak)

“Small, restricted gestures — nervous or constrained.”
Hands stay close to body
Minimal arm extension
Reduces perceived enthusiasm
Common in shy or stressed speakers

4. ≤ 0.30 — Flat or closed gesture amplitude (Poor)

“Arms barely move — low presence, low energy.”
No meaningful gestures
Conveys monotony or lack of conviction
Hard to sustain audience attention
Strong indicator of under-engagement

## 🖐️ Global Gesture Magnitude Score Interpretation Guide

This score assesses the **overall amplitude (size and scope) of hand and arm movements** relative to the torso. It is a weighted average (50/50) of the **Absolute Score** (comparison to an external moderate baseline of 0.20) and the average **Relative Score** (consistency against the subject's own video mean).

**Score Context:**
* The score is bounded between **0 and 1**.
* **A higher score indicates larger, more dynamic, and more consistent arm movements.** 

| Score Range | Behavioral Category | Summary and Impact | Key Behavioral Indicators |
| :---: | :--- | :--- | :--- |
| **$\geq 0.55$** | **Strong Expressive Range (Excellent)** | **"Open, dynamic gestures that project clarity and energy."** The average gesture size is large and the expressive range is high (both absolute size and consistency are good). | * Wide, confident arm movements.* Gestures match the speech rhythm.* High perceived engagement and charisma.* **Common in professional or high-impact communication.** |
| **$0.45 – 0.55$** | **Healthy Expressive Amplitude (Good)** | **"Balanced gestures—visible but not distracting."** The individual gestures appropriately, often meeting the moderate expressive baseline without being overly dramatic. | * Natural arm extension.* Helps structure the message.* Neither too small nor overwhelming.* **Ideal for pitches, meetings, and interviews.** |
| **$0.30 – 0.45$** | **Under-Expressive or Tight Range (Weak)** | **"Small, restricted gestures—nervous or constrained."** Gestures are noticeably smaller than the moderate baseline, suggesting low enthusiasm or physical restraint. | * Hands stay close to the body (pocket-sized gestures).* Minimal arm extension.* Reduces perceived enthusiasm.* **Common in shy or stressed speakers.** |
| **$\leq 0.30$** | **Flat or Closed Gesture Amplitude (Poor)** | **"Arms barely move—low presence, low energy."** The individual shows very little movement, falling significantly below the moderate baseline. | * No meaningful gestures.* Conveys monotony or lack of conviction.* Hard to sustain audience attention.* **Strong indicator of under-engagement or rigidity.** |

## 5.2 - GESTURE ACTIVITY RATE

In [20]:
# --- GESTURE ACTIVITY ---
BASELINE_GESTURE_ACTIVITY_OPTIMAL = 0.02
# Ideal moderate activity rate (not dead, not chaotic).

BASELINE_GESTURE_ACTIVITY_VAR = 0.0004
# Width of Gaussian = how forgiving the model is.

In [21]:
# ========== GESTURE ACTIVITY ==========
# RELATIVE score
z_act = (df_act_5s["value"] - df_act_5s["value"].mean()) / df_act_5s["value"].std()
df_act_5s["rel_score"] = 1 / (1 + np.exp(-z_act))

# ABSOLUTE score
act_abs = df_act_5s["value"].mean()
abs_act_score = np.exp(-((act_abs - BASELINE_GESTURE_ACTIVITY_OPTIMAL)**2) / BASELINE_GESTURE_ACTIVITY_VAR)
# Gaussian preference around ideal activity

# FINAL ACTIVITY SCORE
global_gesture_activity = 0.5 * abs_act_score + 0.5 * df_act_5s["rel_score"].mean()
print (global_gesture_activity)

0.7511591701750326


## Gesture Activity Rate
The $\mathbf{global\_gesture\_activity}$ score emphasizes the quality of the activity by penalizing extremes (too fast or too slow), unlike magnitude where "more" was always better for the absolute component.

(How much movement the hands produce — speed)
Score = sigmoid(-z) → higher = energetic, good

$\geq 0.75$Optimally Active & Dynamic (Excellent)"Perfectly paced gestures with high dynamic emphasis." Overall speed is near the optimal $0.02$ baseline AND the activity level changes dramatically for emphasis (high relative score).* Hand movements are clearly visible but stop when not in use.* Activity pulses dynamically to support speech rhythm.* High confidence and intentional movement.

≥ 0.55 — High gesture dynamics (Excellent)

“Energetic, lively hand activity that enhances expression.”
Matches speech cadence perfectly
Gives rhythm and punch to key points
Strongly boosts engagement

0.45 – 0.55 — Balanced movement (Good)

“Hands move naturally and fluidly.”
Comfortable pace
Good synchrony with voice
Ideal for business communication

0.30 – 0.45 — Slow or hesitant activity (Weak)

“Hands move, but inconsistently or too little.”
Often due to stress or overthinking
Can appear stiff or overly formal
Reduces emotional impact

≤ 0.30 — Almost no gesture activity (Poor)

“Hands passive — low energy, low presence.”
Very little movement
Risk of appearing monotone
Audience engagement drops significantly

## 🏃 Global Gesture Activity Score Interpretation Guide

This score assesses the **overall speed and frequency of arm movements**, aiming to find a balance between static hands and excessive, distracting movement.

**Score Context:**
* The score is bounded between **0 and 1**.
* **A high score indicates activity that is both optimally moderate in speed and highly dynamic (changing) over time.**

| Score Range | Behavioral Category | Summary and Impact | Key Behavioral Indicators |
| :---: | :--- | :--- | :--- |
| **$\geq 0.75$** | **Optimally Active & Dynamic (Excellent)** | **"Perfectly paced gestures with high dynamic emphasis."** Overall speed is near the optimal $0.02$ baseline AND the activity level changes dramatically for emphasis (high relative score). | * Hand movements are clearly visible but stop when not in use.* Activity pulses dynamically to support speech rhythm.* High confidence and intentional movement. |
| **$0.55 – 0.75$** | **Healthy Activity or Highly Consistent (Good)** | **"Appropriate energy level, or highly consistent activity."** This range is achieved by either near-optimal activity that is slightly static OR slightly too high/low activity that is very dynamic. | * Activity may be slightly too high (energetic) or slightly too low (calm).* Dynamic changes are present, signaling transitions or emphasis.* **Overall, the activity is effective and non-distracting.** |
| **$0.35 – 0.55$** | **Suboptimal Activity or Low Dynamism (Weak)** | **"Activity is either too high/low or too monotonic."** Overall activity level is clearly outside the ideal moderate range AND/OR the activity level rarely changes (low relative score). | * Hands are either too still (dead presence) or fidgety (distracting).* Low energy variation makes the presentation feel flat or stiff.* **Indicates potential rigidity or anxiety.** |
| **$\leq 0.35$** | **Extreme Activity or Static (Poor)** | **"Chaotic or near-static movement, significantly suboptimal."** The overall activity is far from the $0.02$ optimum, severely penalizing the score. | * Constant, high-speed movement (jitter, fidgeting) or almost complete lack of movement.* Conveys nervousness, distraction, or extreme lack of engagement.* **Activity actively detracts from communication.** |

## 5.3 - GESTURE JITTER (more = chaotic = BAD)

In [22]:
BASELINE_GESTURE_JITTER_OPTIMAL = 0.0005
BASELINE_GESTURE_JITTER_RANGE   = 0.0004

In [23]:
# ========== GESTURE JITTER (absolute + relative) ==========
z_jit = (df_jitter_5s["value"] - df_jitter_5s["value"].mean()) / df_jitter_5s["value"].std()
df_jitter_5s["rel_score"] = 1 / (1 + np.exp(z_jit))   # inverted

# ABSOLUTE score
jitter_abs = df_jitter_5s["value"].mean()
abs_gesture_jitter_score = 1 / (
    1 + np.exp((jitter_abs - BASELINE_GESTURE_JITTER_OPTIMAL) / BASELINE_GESTURE_JITTER_RANGE)
)

# FINAL score
global_gesture_jitter = 0.5 * abs_gesture_jitter_score + 0.5 * df_jitter_5s["rel_score"].mean()
print(global_gesture_jitter)

0.5543208722341544


(Instability or shakiness in the hands)
Score = sigmoid(+z) → higher = more stable (better)

≥ 0.55 — Extremely stable gestures (Excellent)

“Smooth, intentional movement without fidgeting.”
Gestures appear deliberate
No micro-shaking
Indicates strong composure and control

0.45 – 0.55 — Natural stability (Good)

“Healthy fluidity — no distracting instability.”
Small corrective motions only
Low perceptible jitter
Ideal for professional speaking

0.30 – 0.45 — Mild instability (Weak)

“Occasional shaking or unintentional flicks.”
Visible nervous energy
Hands adjust too frequently
Subtle distraction to the audience

≤ 0.30 — Strong jitter / instability (Poor)

“Fidgety, shaky, restless hands.”
Chaotic or uncontrolled movement
Destroys composure
Clear sign of anxiety or lack of grounding

## 🌪️ Global Gesture Jitter Score Interpretation Guide

This score assesses the **smoothness and consistency of arm and hand speed**. It is highly sensitive to rapid, uncontrolled changes in velocity. The final score is a weighted average (50/50) of the **Absolute Score** (comparison to an external smoothness baseline of 0.0005) and the average **Relative Score** (temporal consistency of that smoothness).

**Score Context:**
* The score is bounded between **0 and 1**.
* **A higher score indicates greater smoothness (less jitter).** The scoring logic is **inverted**; lower raw jitter leads to a higher score.

| Score Range | Behavioral Category | Summary and Impact | Key Behavioral Indicators |
| :---: | :--- | :--- | :--- |
| **$\geq 0.75$** | **High Fluidity & Control (Excellent)** | **"Movements are highly controlled, smooth, and intentional."** Jitter is significantly below the optimal threshold, demonstrating mastery over movement. | * Gestures start and finish smoothly without sudden speed changes.* Hands appear steady when at rest or transitioning.* **Projects high confidence, comfort, and composure.** |
| **$0.55 – 0.75$** | **Good Smoothness (Good)** | **"Movements are generally smooth, with only minor, non-distracting jitters."** Overall jitter is close to or below the acceptable $0.0005$ threshold. | * Motion is clear and purposeful.* Any slight jitters are not sustained or visible enough to distract the viewer.* **Suggests a calm, professional delivery.** |
| **$0.35 – 0.55$** | **Noticeable Jitter/Inconsistency (Weak)** | **"Movement speed is unsteady, suggesting lack of control or nervousness."** Jitter is noticeably higher than the optimal $0.0005$ threshold, creating visible roughness in motion. | * Hands appear to 'flick' or tremble at rest or during pauses.* Arm movements show obvious, rapid changes in speed (start/stop).* **May indicate anxiety, low preparation, or discomfort.** |
| **$\leq 0.35$** | **Excessive Jitter/Roughness (Poor)** | **"Highly erratic, jerky, or chaotic movement that disrupts attention."** Jitter is far above the acceptable threshold, severely penalizing the score. | * Frequent, small, involuntary movements (fidgeting).* Gestures appear rushed, uncontrolled, or poorly coordinated.* **Actively detracts from the message and conveys high stress.** |

## 5.4 - BODY SWAY (more sway = BAD)

In [24]:
# --- BODY SWAY ---
BASELINE_BODY_SWAY_SCALE = 3000
# Converts tiny displacements into human-readable quality.
# Sway < 0.001 → score close to 1.0.

In [25]:
# ========== BODY SWAY ==========
# RELATIVE score
z_sway = (df_sway_5s["value"] - df_sway_5s["value"].mean()) / df_sway_5s["value"].std()
df_sway_5s["rel_score"] = 1 / (1 + np.exp(z_sway))   # inverted

# ABSOLUTE score
global_sway_abs = df_sway_5s["value"].mean()
abs_sway_score = np.exp(-global_sway_abs * BASELINE_BODY_SWAY_SCALE)
# almost no sway → ~1.0

# FINAL SWAY SCORE
global_body_sway = 0.5 * abs_sway_score + 0.5 * df_sway_5s["rel_score"].mean()
print (global_body_sway)

0.7234791750440199


(Instability of torso movement)
Score = sigmoid(+z) → higher = more stable (better)

≥ 0.55 — Grounded, controlled posture (Excellent)

“Torso remains stable and solid.”
Project authority and confidence
Excellent weight distribution
No swaying or rocking
Often seen in trained public speakers

0.45 – 0.55 — Natural controlled movement (Good)

“Small natural shifts, nothing distracting.”
Appears relaxed and comfortable
Aligned with natural breathing
Perfectly acceptable in presentations

0.30 – 0.45 — Restless torso movement (Weak)

“Slight swaying, shifting, or rocking.”
Visible uncertainty or stress
Reduces perceived assertiveness
Common among anxious speakers

≤ 0.30 — Strong body sway (Poor)

“Frequent rocking or shifting weight.”
Highly distracting
Signals nervousness
Significantly reduces presence and credibility

## 🧍 Global Body Sway Score Interpretation Guide

This score assesses the **stability and stillness of the speaker's torso**, specifically measuring the **inconsistency or variability** of torso movement (sway variance). The score is heavily inverted, utilizing an **Exponential Decay** function for the Absolute Score, which severely penalizes any significant movement away from the ideal of zero sway.

**Score Context:**
* The score is bounded between **0 and 1**.
* **A higher score indicates greater stability and stillness (less sway).** 

| Score Range | Behavioral Category | Summary and Impact | Key Behavioral Indicators |
| :---: | :--- | :--- | :--- |
| **$\geq 0.75$** | **High Stability & Composure (Excellent)** | **"Torso is rock-steady; movement is minimal, controlled, and intentional."** The mean sway is near zero, and the stance is consistently stable. | * Speaker appears grounded and highly centered.* Torso position rarely changes, maximizing focus on upper body/face.* **Projects high confidence, comfort, and authority.** |
| **$0.55 – 0.75$** | **Good Stability (Good)** | **"Stance is mostly stable, with only minor, smooth, non-distracting shifts."** The mean sway is slightly above zero, but not highly erratic. The stability is acceptable. | * Body shifts are slow and deliberate (e.g., shifting weight once or twice).* Minimal or no distracting lateral torso movement.* **Suggests a calm, professional presence.** |
| **$0.35 – 0.55$** | **Noticeable Unsteadiness (Weak)** | **"Uncontrolled or excessive sway is present, distracting the viewer."** Torso movement is high enough to trigger the steep penalty from the exponential decay function. | * Frequent, small, shifting movements (fidgeting with feet/torso).* Lateral movement of the torso is obvious or erratic.* **May indicate anxiety, restlessness, or low physical comfort.** |
| **$\leq 0.35$** | **Severe Sway or Unstable (Poor)** | **"Highly erratic movement or continuous, large shifts in the torso."** The sway variance is far from the zero ideal, resulting in a very low score. | * Constant visible rocking or lateral movement of the body.* Stance appears loose, nervous, or unprofessional.* **Actively detracts from communication and conveys high stress.** |

## 5.5 - POSTURE OPENNESS (more open = good)

In [26]:
# --- POSTURE OPENNESS BASELINE ---
BASELINE_POSTURE_OPTIMAL = 120
# 120° is typical for “open” posture (shoulders relaxed, chest open).

BASELINE_POSTURE_RANGE = 15
# Wider = less aggressive scoring.

In [27]:
# RELATIVE score
z_open = (df_open_5s["value"] - df_open_5s["value"].mean()) / df_open_5s["value"].std()
df_open_5s["rel_score"] = 1 / (1 + np.exp(-z_open))

# ABSOLUTE score
open_abs = df_open_5s["value"].mean()
abs_open_score = 1 / (1 + np.exp(-(open_abs - BASELINE_POSTURE_OPTIMAL) / BASELINE_POSTURE_RANGE))

# FINAL OPENNESS SCORE
global_posture_openness = 0.5 * abs_open_score + 0.5 * df_open_5s["rel_score"].mean()
print (global_posture_openness)

0.7383540295243438


(How open or closed the chest / shoulders are)
Score = sigmoid(-z) → higher = more open (better)

≥ 0.55 — Open, expansive posture (Excellent)

“Strong, confident posture — welcoming and authoritative.”
Shoulders open
Chest is naturally lifted
Maximizes presence & audience connection
Highly persuasive posture

0.45 – 0.55 — Healthy neutral openness (Good)

“Comfortably open posture — approachable and relaxed.”
Not too tight, not too dramatic
Solid baseline posture
Ideal for interviews, panels, pitches

0.30 – 0.45 — Closed or slightly constricted posture (Weak)

“Shoulders turning inward — slight defensiveness.”
Reduces presence
Signals lack of confidence
Subtle barrier to audience connection

≤ 0.30 — Closed, collapsed posture (Poor)

“Strong inward rotation of shoulders.”
Protective / stressed body language
Low authority and low perceived competence
Strong negative impact on credibility

## 🧘 Global Posture Openness Score Interpretation Guide

This score assesses the **degree of relaxation and openness** in the speaker's shoulders and chest, translating to perceived confidence and comfort. It is based on the angle formed by the shoulders and the neck center, rewarding angles $\mathbf{> 120^\circ}$ (the optimal baseline).

**Score Context:**
* The score is bounded between **0 and 1**.
* **A higher score indicates a more open, relaxed, and confident posture.**

| Score Range | Behavioral Category | Summary and Impact | Key Behavioral Indicators |
| :---: | :--- | :--- | :--- |
| **$\geq 0.75$** | **Highly Open & Relaxed (Excellent)** | **"Posture is consistently expansive, demonstrating high comfort and presence."** The average shoulder angle is significantly $\mathbf{> 120^\circ}$. | * Shoulders are visibly relaxed and back.* Chest is open and prominent.* **Projects high confidence, comfort, and authenticity.** |
| **$0.55 – 0.75$** | **Acceptably Open (Good)** | **"Posture is generally open, meeting or slightly exceeding the $120^\circ$ ideal."** The speaker maintains a healthy, non-defensive stance. | * Stance is natural; shoulders are neither visibly hunched nor overly rigid.* Slight variation in openness (breathing, minimal movement).* **Suggests engagement and a professional level of composure.** |
| **$0.35 – 0.55$** | **Slightly Closed/Hunched (Weak)** | **"Posture is consistently below the $120^\circ$ baseline, indicating slight tension or disinterest."** The average angle is below the ideal, pulling the absolute score down. | * Shoulders are slightly rolled forward.* Torso appears rigid or slightly compressed.* **May signal mild nervousness, lack of enthusiasm, or a defensive stance.** |
| **$\leq 0.35$** | **Closed or Rigid Posture (Poor)** | **"Posture is noticeably hunched, rigid, or defensive throughout the presentation."** The angle is significantly low, resulting in a very low score. | * Shoulders are clearly rolled forward and closed (hunched).* Speaker appears small, uncomfortable, or physically restricted.* **Strongly suggests high tension, insecurity, or reluctance.** |

====================================================
# 6 - MERGING + global scoring
====================================================

In [28]:
df_mag_5s = df_mag_5s.rename(columns={
    "value": "value_gesture_magnitude",
    "rel_score": "rel_score_gesture_magnitude"
})

df_act_5s = df_act_5s.rename(columns={
    "value": "value_gesture_activity",
    "rel_score": "rel_score_gesture_activity"
})

df_jitterbody_5s = df_jitter_5s.rename(columns={
    "value": "value_gesture_jitter",
    "rel_score": "rel_score_gesture_jitter"
})

df_sway_5s = df_sway_5s.rename(columns={
    "value": "value_body_sway",
    "rel_score": "rel_score_body_sway"
})

df_open_5s = df_open_5s.rename(columns={
    "value": "value_posture_openness",
    "rel_score": "rel_score_posture_openness"
})

df_body_merged = (
    df_mag_5s[["start_sec", "end_sec", "value_gesture_magnitude", "rel_score_gesture_magnitude"]]
    .merge(df_act_5s[["start_sec", "end_sec", "value_gesture_activity", "rel_score_gesture_activity"]], on=["start_sec", "end_sec"])
    .merge(df_jitterbody_5s[["start_sec", "end_sec", "value_gesture_jitter", "rel_score_gesture_jitter"]], on=["start_sec", "end_sec"])
    .merge(df_sway_5s[["start_sec", "end_sec", "value_body_sway", "rel_score_body_sway"]], on=["start_sec", "end_sec"])
    .merge(df_open_5s[["start_sec", "end_sec", "value_posture_openness", "rel_score_posture_openness"]], on=["start_sec", "end_sec"])
)

df_body_merged.index = [f"window_{i}" for i in range(len(df_body_merged))]
df_body_merged


,start_sec,end_sec,value_gesture_magnitude,rel_score_gesture_magnitude,value_gesture_activity,rel_score_gesture_activity,value_gesture_jitter,rel_score_gesture_jitter,value_body_sway,rel_score_body_sway,value_posture_openness,rel_score_posture_openness
window_0,0,5,0.503205,0.594465,0.026740,0.859621,0.000452,0.285382,0.000031,0.208100,180.0,0.916519
window_1,1,6,0.516345,0.644346,0.026396,0.846857,0.000445,0.297338,0.000031,0.210664,180.0,0.905735
window_2,2,7,0.533183,0.703848,0.022136,0.609322,0.000276,0.634361,0.000020,0.497335,180.0,0.805464
window_3,3,8,0.530507,0.694778,0.021688,0.577186,0.000309,0.568221,0.000020,0.514489,180.0,0.739559
window_4,4,9,0.519935,0.657492,0.023106,0.675369,0.000350,0.482763,0.000020,0.495033,180.0,0.458645
window_5,5,10,0.504605,0.599895,0.023560,0.704230,0.000357,0.467652,0.000021,0.488102,180.0,0.356939
window_6,6,11,0.521935,0.664717,0.024073,0.734942,0.000596,0.106751,0.000027,0.290602,180.0,0.282226
window_7,7,12,0.496777,0.569264,0.026642,0.856093,0.000644,0.074014,0.000026,0.328330,180.0,0.199619
window_8,8,13,0.454970,0.402498,0.027117,0.872617,0.000666,0.062694,0.000025,0.350458,180.0,0.277615
window_9,9,14,0.435162,0.328633,0.025120,0.790986,0.000603,0.101357,0.000024,0.369116,180.0,0.341345


In [29]:
def compute_body_score(
    global_mag,
    global_act,
    global_jit,
    global_sway,
    global_open
):
    """
    Final body-language score = equal-weight composite across 5 metrics.
    """

    return (
        global_mag +
        global_act +
        global_jit +
        global_sway +
        global_open
    ) / 5

print(compute_body_score(global_gesture_magnitude, global_gesture_activity, global_gesture_jitter, global_body_sway, global_posture_openness))

0.6965105475023418
